# import libraries

In [1]:
import pandas as pd

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

# ignore warning 
import warnings
warnings.filterwarnings("ignore")

# upload data

In [2]:
df1 = pd.read_json('/fastcampus-data/select_column_version_3.json')
df = df1.copy()

In [3]:
# log data의 기간(4개월)과 동일한 기간의 데이터 사용하기 위해 start date 지정 
start_date = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]

# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > start_date]

# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
complete_df = complete_df.reset_index(drop=True)

# 도서 카테고리만 가져오기
df_book = complete_df[complete_df['name'] == '도서']
df_book = df_book.sort_values(by='date_paid')


# train test split
최신성을 반영하기 위해 전체 데이터의 기간(4개월) 중 마지막 3주의 데이터를 test로 사용했다.


4개월의 데이터를 활용하여 3주 이전의 데이터로 학습하고 마지막 3주를 예측한다. 

In [4]:
from datetime import datetime, timedelta

# 마지막 3주를 test data로 사용
date = df_book['date_paid'].max()-timedelta(weeks=3)
train_before_preprocess = df_book[df_book['date_paid'] < date]
test_before_preprocess = df_book[df_book['date_paid'] >= date]

# train 1개 이상 구매 기록유저에 대한 학습 진행
train_drop_row_index = train_before_preprocess['customer_id'].value_counts()[train_before_preprocess['customer_id'].value_counts()>=1].index
train = train_before_preprocess[train_before_preprocess['customer_id'].isin(train_drop_row_index)]

In [5]:
# 전처리할 test index 확인: 2번 이상 구매한 고객만 추출
drop_row_index = test_before_preprocess['customer_id'].value_counts()[test_before_preprocess['customer_id'].value_counts()>2].index

# 최근 구매 3개 기록을 test 사용
test_list = []
for idx in drop_row_index:
    test_list.append(test_before_preprocess[test_before_preprocess['customer_id']==idx].iloc[:])
test = pd.concat(test_list)

# train 있는 유저에 대해서만 진행
test = test[test['customer_id'].isin(train['customer_id'])]

In [6]:
train.date_paid = pd.to_datetime(train.date_paid)

In [7]:
# date_paid에 날짜만 남기기
train.date_paid = train.date_paid.apply(lambda x: x.date)

# train test eda

### 전처리 전후 비교
test의 경우 2번 이상 구매한 유저에 대해서만 테스트 가능  
전처리 후의 데이터 수 비교

In [8]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 5042 train 전처리 후: 5042


In [9]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 889 test 전처리 후: 116


### user 수 비교

In [10]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 2323


In [11]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 29


### item 개수 비교

In [12]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 243 test 아이템 수 : 67


In [13]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 180


In [14]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 4


In [ ]:
print(train.shape)
print(test.shape)

(5042, 16)
(116, 16)


# Load Transactions

In [ ]:
train = train[['date_paid','customer_id','name_x','product_ids']]
print('Train shape:',train.shape)

In [ ]:
# 한 유저당 구매한 제품들의 unique -> 중복을 없앰.
train_apri = train.groupby('customer_id')['name_x'].unique().reset_index()
train_apri[:30]

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
pd.set_option('display.max_colwidth', -1)
transactions = [a[1]['name_x'].tolist() for a in list(train.groupby(['customer_id','date_paid']))]

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# Convert product list per customer_id into one-hot

In [ ]:
orders_1hot = pd.DataFrame(te_ary, columns=te.columns_)
pf = orders_1hot.describe()

describe를 이용하여 most popular item 확인

In [ ]:
f = pf.iloc[0]-pf.iloc[3] # count - frequent : f가 작을 수록 판매된 횟수가 많은 것
a = f.tolist()
b = list(f.index)
item = pd.DataFrame([[a[r],b[r]]for r in range(len(a))], columns=['Count','Item'])
item = item.sort_values(['Count'], ascending=False).head(10)
print("MOST POPULAR")
item

# Apriori

In [ ]:
from mlxtend.frequent_patterns import apriori

In [ ]:
is_ap = apriori(orders_1hot, min_support=0.01, max_len=3, use_colnames=True)
is_ap.sort_values(by = ['support'], ascending = False)

# Association Rules
confidence로 threshold를 설정하여 filtering한 다음 lift값을 기준으로 sorting한다.

In [ ]:
from mlxtend.frequent_patterns import association_rules

# matrix의 default가 confidence로 되어있어 따로 지정해주지 않아도 됨.
rules_confidence_item = association_rules(is_ap, min_threshold=0.04)

#lift 기준으로 sort
rules_confidence_item = rules_confidence_item.sort_values(['lift'], ascending = False)
display(rules_confidence_item)

### 결론: confidence가 높은 association rule의 개수가 적다.
-> 연관성 추천이 불가능하다.

# 예측값이 실제 구매내역에 있는지 확인
association rule의 antecedents와 consequents에 해당하는 item을 이용하여 연관 추천

In [ ]:
apriori_items = rules_confidence_item[['antecedents','consequents']].iloc[:15]

apriori_items['antecedents'] = apriori_items['antecedents'].apply(lambda x: list(map(str,x))[0])
apriori_items['consequents'] = apriori_items['consequents'].apply(lambda x: list(map(str,x))[0])

apriori_items.rename(columns={"antecedents": "A", "consequents": "A의 연관 아이템 (B)"})

In [ ]:
train_items = train[['date_paid','customer_id','name_x','product_ids']]

# 예측값을 담을 column
train_items['apriori_pred'] = train_items['name_x'].apply(lambda x: apriori_items[apriori_items['antecedents']==x]['consequents'].values[0] if x in list(apriori_items['antecedents']) \
                            else None)

In [ ]:
name_x_df = train_items.groupby('customer_id')['name_x'].unique().reset_index()
apriori_pred_df  = train_items.groupby('customer_id')['apriori_pred'].unique().reset_index()
real_pred = test.groupby('customer_id')['name_x'].unique().reset_index().rename(columns={'name_x':'real_product'})

train_pred = pd.merge(name_x_df,apriori_pred_df, on='customer_id', how='left')
train_real_pred = pd.merge(train_pred,real_pred, on='customer_id', how='right')

train_real_pred[['name_x','apriori_pred','real_product']]

In [ ]:
# 아이템 맞춘 개수
cnt = 0
for preds, reals in zip(train_real_pred['apriori_pred'], train_real_pred['real_product']):
    for pred in preds:
        if pred in reals:
            cnt += 1
print('예측 성공한 아이템 개수: ',cnt)